# pneumonia detection from chest x-rays  with transfer learning (mobilenetv2)

## dependencies


In [9]:
import os
import tensorflow

## dataset


In [ ]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia
!unzip chest-xray-pneumonia.zip

In [17]:
!mv chest_xray/chest_xray /content/chest_xray_clean

In [18]:
!find /content/chest_xray_clean -name "__MACOSX" -exec rm -rf {} +
!find /content/chest_xray_clean -name ".DS_Store" -delete

In [21]:
!rm -rf /content/chest_xray

In [22]:
base = "/content/chest_xray_clean"
for split in ["train", "test", "val"]:
    split_dir = os.path.join(base, split)
    classes = [c for c in os.listdir(split_dir) if os.path.isdir(os.path.join(split_dir, c))]
    counts = {c: len(os.listdir(os.path.join(split_dir, c))) for c in classes}
    print(split, counts)

train {'NORMAL': 1341, 'PNEUMONIA': 3875}
test {'NORMAL': 234, 'PNEUMONIA': 390}
val {'NORMAL': 8, 'PNEUMONIA': 8}
